## Import Libraries

In [1]:
from google.colab import drive
import os

import pandas as pd
import numpy as np

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/USP/2022_1º/ACH2016 IA/dados/DADOS/')

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Load data

In [2]:
# Load data
# https://download.inep.gov.br/microdados/microdados_enem_2020.zip

df = pd.read_csv('MICRODADOS_ENEM_2020.csv',encoding='ISO-8859-1', sep=';', dtype=str)[['TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024', 'Q025', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC', 'TX_GABARITO_MT']]
df.dropna(inplace=True)

## Adapt dataset

In [ ]:
# Calcular quantidade de acertos
def calcular(a,b):
    return sum (a[i] == b[i] for i in range(len(a)))

df['CN_ACERTOS'] = df.apply(lambda x: calcular(x['TX_RESPOSTAS_CN'], x['TX_GABARITO_CN']), axis=1)
df['CH_ACERTOS'] = df.apply(lambda x: calcular(x['TX_RESPOSTAS_CH'], x['TX_GABARITO_CH']), axis=1)
df['LC_ACERTOS'] = df.apply(lambda x: calcular(x['TX_RESPOSTAS_LC'], x['TX_GABARITO_LC']), axis=1)
df['MT_ACERTOS'] = df.apply(lambda x: calcular(x['TX_RESPOSTAS_MT'], x['TX_GABARITO_MT']), axis=1)
df.drop(columns=['TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC', 'TX_GABARITO_MT'], inplace=True)


# Calcular quantidade total de acertos
df['Total_acertos'] = df.apply(lambda x: x['CN_ACERTOS']+x['CH_ACERTOS']+x['LC_ACERTOS']+x['MT_ACERTOS'], axis=1)
df['Total_acertos_cat'] = df.apply(lambda x: x['Total_acertos'] // 10, axis=1)


df['TP_SEXO'] = df['TP_SEXO'].map({'F': 1, 'M': 0})

df.to_csv('dados-def.csv', encoding='ISO-8859-1', sep=';', index=False)

In [ ]:
df = pd.read_csv('dados-def.csv',encoding='ISO-8859-1', sep=';', dtype=str)

# acho q n pode fazer assim
questoes = ['Q001', 'Q002', 'Q003', 'Q004', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024', 'Q025']
for qst in questoes:
    print(qst)
    df[qst] = df[qst].map({'A': 1,  'B': 2,  'C': 3,  'D': 4,  'E': 5,  'F': 6,  'G': 7,  'H': 8,  'I': 9,  'J': 10,  'K': 11,  'L': 12,  'M': 13,  'N': 14,  'O': 15,  'P': 16,  'Q': 17})

print(df)


# Precisa normalizar ?
# Dividir coluna numerica e categorica

## Train, Test

In [27]:
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import KFold

df = df.iloc[:5000] # so para o train n ser demorado

y = df['Total_acertos_cat']
features = ['TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024', 'Q025', 'Total_acertos_cat']
X = df[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import precision_recall_fscore_support

def report(tam, y_test, y_pred):
    print(" ("+str(tam).ljust(2, ' ')+") ",end='')
    p,r,f,_none = precision_recall_fscore_support(y_test, y_pred, average='macro')
    print('  P =',"{:2.2f}".format(p).replace(".",","),end='')
    print('  R =',"{:2.2f}".format(r).replace(".",","),end='')
    print('  F =',"{:2.2f}".format(f).replace(".",","))


# Train
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)


# Test
y_pred = neigh.predict(X_test)
report(X_test.shape[1], y_test, y_pred)